In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [3]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_leaderboard.npz")

In [4]:
URM_test.shape

(4439758, 23692)

In [5]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [6]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [7]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [8]:
is_sorted(Session_ids)

True

In [9]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/leaderboard/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
1520919
loaded UCF
0
loading ICBF
1485908
loaded ICBF
0
loading Graph
864863
loaded Graph
0
loading ICF
1815420
loaded ICF
0
loading UCF_W
1483285
loaded UCF_W
0
loading ICBF_W
1485908
loaded ICBF_W
0
loading Graph_W
910517
loaded Graph_W
0
loading ICF_W
1891120
loaded ICF_W
0
loading UCF_WW
1346987
loaded UCF_WW
0
loading ICBF_WW
1485908
loaded ICBF_WW
0
loading Graph_WW
910517
loaded Graph_WW
0
loading ICF_WW
1891120
loaded ICF_WW


In [10]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids

In [11]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [12]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [13]:
import ast
submission_df=pd.read_csv("../dataset/candidates/NN/GRU/leaderboard/candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [14]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/leaderboard/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [15]:
import ast
submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/leaderboard/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [16]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [17]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [18]:
dataset.fillna(0,inplace=True)

In [19]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_leaderboard.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items'])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [20]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [21]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,Score_UCF_WW,...,Max_Score_ICF_WW,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,21,0.099430,0.0,0.031298,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34,0.035716,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52,0.182064,0.0,0.097918,2.199146e-02,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,80,0.053687,0.0,0.034619,1.430049e-02,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126,0.037062,0.0,0.025345,1.976426e-08,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
gc.collect()

111

In [23]:
dataset=dataset.sort_values("Session_Id")

In [24]:
gc.collect()

46

In [25]:
import lightgbm

In [26]:
k=10

In [27]:
result = []

In [28]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'../rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))
    # dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))

#combine scores
result = np.array(result)
result

array([[-0.2345142 , -0.15051161, -0.15051161, ..., -0.32809278,
        -0.32809278, -0.19619684],
       [-0.84287002, -0.88355346, -0.88355346, ..., -0.74774853,
        -0.74774853,  0.01319398],
       [-0.11524855, -0.11524855, -0.11524855, ..., -0.20596967,
        -0.20596967,  0.00376855],
       ...,
       [-1.16127238, -1.12320008, -1.12320008, ..., -0.88578129,
        -0.88578129, -0.52037868],
       [-2.22874402, -2.28751585, -2.28751585, ..., -2.31294779,
        -2.31294779, -0.63536197],
       [-2.3434682 , -2.29119361, -2.29119361, ..., -2.23731285,
        -2.23731285, -0.3739293 ]])

In [29]:
result.shape

(10, 5969611)

In [30]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [31]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

67

In [32]:
dataset.head()

,Session_Id,Item_ID,score
2873233,26,3,-1.060284
4356528,26,304,-1.046497
4356527,26,291,-1.046497
4356526,26,287,-1.046497
4356525,26,256,-1.046497


In [33]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [34]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [35]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [36]:
dataset.iloc[0]

Session_Id                                                   26
Item_ID       [3, 304, 291, 287, 256, 241, 207, 153, 105, 33...
score         [-1.0602842961359573, -1.046497079485047, -1.0...
indices       [632, 438, 327, 328, 329, 330, 331, 332, 333, ...
Name: 0, dtype: object

In [37]:
dataset.iloc[0]["Item_ID"][536]

3410

In [38]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [39]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [40]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [41]:
dataset

,Session_Id,predictions,rank
0,26,"[606, 4884, 2852, 746, 1227, 2814, 865, 2572, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[772, 270, 4878, 1827, 3676, 3633, 3400, 4024,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[3398, 1659, 734, 1795, 1653, 2997, 4911, 2037...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[3954, 4938, 392, 3288, 4900, 3289, 3291, 4987...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[2175, 1004, 143, 2995, 114, 4784, 4785, 4868,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
9995,892056,"[3678, 1465, 1105, 3422, 4418, 2785, 4138, 335...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9996,892163,"[3706, 736, 1658, 1446, 1206, 4981, 1926, 2408...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9997,892397,"[1558, 971, 686, 2990, 4826, 2251, 1539, 1041,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9998,892442,"[4489, 4657, 1463, 2639, 2674, 3333, 4418, 195...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [42]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [43]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [44]:
dataset.columns=["session_id","code","rank"]

In [45]:
dataset.head()

,session_id,code,rank
0,26,"(606, 4884, 2852, 746, 1227, 2814, 865, 2572, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(772, 270, 4878, 1827, 3676, 3633, 3400, 4024,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(3398, 1659, 734, 1795, 1653, 2997, 4911, 2037...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(3954, 4938, 392, 3288, 4900, 3289, 3291, 4987...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(2175, 1004, 143, 2995, 114, 4784, 4785, 4868,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [46]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [47]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [48]:
print(len(submission_df))

1000000


In [49]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   session_id  1000000 non-null  int32
 1   item_id     1000000 non-null  int64
 2   rank        1000000 non-null  int32
dtypes: int32(2), int64(1)
memory usage: 22.9 MB


In [50]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [51]:
submission_df.to_csv(
    f'../leaderboard_sub.csv',
    index=False
    )